In [8]:
from selenium import webdriver

from selenium.webdriver.common.by import By


In [9]:

from datetime import datetime


In [10]:
todays_date = datetime.today()
day_of_month = todays_date.strftime('%d')
date_link = todays_date.strftime('%Y/%m/%d')
date_element = todays_date.strftime('%Y-%m-%d')

In [11]:
from selenium.webdriver.firefox.options import Options
opts = Options()
opts.add_argument("--headless")

In [13]:
browser = webdriver.Firefox(options=opts)
browser.get(f'https://www.nytimes.com/{date_link}/crosswords/spelling-bee-forum.html')

In [14]:
# day + sb before first dash
for e in browser.find_elements(By.ID, f'{day_of_month}sb-forum-{date_element}'):
    print(e.text)

Center letter is in bold.
W C D E K L O
WORDS: 25, POINTS: 107, PANGRAMS: 1 (1 Perfect)
4 5 6 7 8 Σ
C: 1 1 - - - 2
D: - 2 - 1 - 3
L: 1 1 - - 1 3
O: 1 - - - - 1
W: 7 2 5 1 1 16
Σ: 10 6 5 2 2 25
Two letter list:
CO-2
DO-1 DW-2
LE-1 LO-2
OW-1
WE-9 WO-7


In [16]:
# day + sb before first dash
grid_text = browser.find_element(By.ID, f'{day_of_month}sb-forum-{date_element}').text

In [17]:
browser.quit()

In [19]:
# assume that letters are in line 2
# totals are in line 3
# word lengths are in line 4
# grid is line 5-?
# 2LL is ?-?
print(grid_text.split("\n")) 


['Center letter is in bold.', 'W C D E K L O', 'WORDS: 25, POINTS: 107, PANGRAMS: 1 (1 Perfect)', '4 5 6 7 8 Σ', 'C: 1 1 - - - 2', 'D: - 2 - 1 - 3', 'L: 1 1 - - 1 3', 'O: 1 - - - - 1', 'W: 7 2 5 1 1 16', 'Σ: 10 6 5 2 2 25', 'Two letter list:', 'CO-2', 'DO-1 DW-2', 'LE-1 LO-2', 'OW-1', 'WE-9 WO-7']


In [20]:
grid_text_list = grid_text.split("\n")
todays_letters = grid_text_list[1].split()
todays_lengths = grid_text_list[3].split()[:-1]

In [21]:
todays_lengths

['4', '5', '6', '7', '8']

In [22]:
import re

In [27]:
def get_lengths_by_letter(first_letter):
    lengths_dict = {}
    for grid_line in grid_text_list:
        if grid_line[:2] == first_letter + ':':
            # first 3 characters are letter, :, space
            # last character is sum
            lengths = grid_line[3:-1].replace('-', '0').split()
            for i in range(len(todays_lengths)):
                lengths_dict[int(todays_lengths[i])] = int(lengths[i])
            return lengths_dict
        
    for i in range(len(todays_lengths)):
        lengths_dict[int(todays_lengths[i])] = 0
    return lengths_dict
    

In [24]:
def get_bigrams_by_letter(first_letter):
    for grid_line in grid_text_list:
        if re.match(f'{first_letter}[A-Z]-\d\d?', grid_line):
            bigrams = grid_line.split()
            bigram_dict = {}
            for bigram in bigrams:
                bigram_dict[bigram[:2]] = int(bigram[3:])
            return bigram_dict
    return {}


In [30]:
# TODO check total number of words

results_dict = {}
for letter in todays_letters:
    results_dict[letter] = {'lengths': get_lengths_by_letter(letter)}
    results_dict[letter]['bigrams'] = get_bigrams_by_letter(letter)


In [31]:
results_dict

{'W': {'lengths': {4: 7, 5: 2, 6: 5, 7: 1, 8: 1},
  'bigrams': {'WE': 9, 'WO': 7}},
 'C': {'lengths': {4: 1, 5: 1, 6: 0, 7: 0, 8: 0}, 'bigrams': {'CO': 2}},
 'D': {'lengths': {4: 0, 5: 2, 6: 0, 7: 1, 8: 0},
  'bigrams': {'DO': 1, 'DW': 2}},
 'E': {'lengths': {4: 0, 5: 0, 6: 0, 7: 0, 8: 0}, 'bigrams': {}},
 'K': {'lengths': {4: 0, 5: 0, 6: 0, 7: 0, 8: 0}, 'bigrams': {}},
 'L': {'lengths': {4: 1, 5: 1, 6: 0, 7: 0, 8: 1},
  'bigrams': {'LE': 1, 'LO': 2}},
 'O': {'lengths': {4: 1, 5: 0, 6: 0, 7: 0, 8: 0}, 'bigrams': {'OW': 1}}}

In [11]:
get_lengths_by_letter('A')

['2', '1', '0', '2', '2', '0']

In [24]:
get_bigrams('A')

{'AC': 5, 'AI': 1, 'AM': 1}

In [37]:
my_words = """
Academe
Academia
Academic
Academy
Aimed
Amid
Came
Cami
Dame
Dammed
Deem
Deemed
Diadem
Dime
Dimmed
Edema
Emcee
Emceed
Imam
Immediacy
Macadamia
Mace
Maced
Made
Maid
Maim
Maimed
Mama
Mamma
Mayday
Mead
Mecca
Media
Medic
Meme
Memed
Mica
Mice
Midday
Midi
Mime
Mimed
Mimic
"""

In [40]:
print(my_words.split("\n")[1:-1])

['Academe', 'Academia', 'Academic', 'Academy', 'Aimed', 'Amid', 'Came', 'Cami', 'Dame', 'Dammed', 'Deem', 'Deemed', 'Diadem', 'Dime', 'Dimmed', 'Edema', 'Emcee', 'Emceed', 'Imam', 'Immediacy', 'Macadamia', 'Mace', 'Maced', 'Made', 'Maid', 'Maim', 'Maimed', 'Mama', 'Mamma', 'Mayday', 'Mead', 'Mecca', 'Media', 'Medic', 'Meme', 'Memed', 'Mica', 'Mice', 'Midday', 'Midi', 'Mime', 'Mimed', 'Mimic']
